# Redes preentrenadas

Entrenar modelos tan grandes resulta costoso y podemos beneficiarnos de la estructura de los modelos existente de forma que solo ajustemos los parámetros finales de cara a la tarea en cuestión que queramos resolver.

![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el [Hands on Machine Learning for Python](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/ch14.html#cnn_chapter).
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Las redes se pueden incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

base_model = ResNet50V2(include_top= True, input_shape=(224, 224, 3), weights='imagenet', classifier_activation= 'softmax')

In [ ]:
len(base_model.layers)

In [ ]:
base_model.summary()

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [ ]:
import os
import cv2
import numpy as np
from skimage.io import imread

def read_data(path):
    X = []
    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224,224))
        print(path + '/' + file)

        X.append(smallimage)
    return np.array(X)

In [ ]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input

X_test = read_data('data/muestras')

#Preprocesar las imágenes tal y como entran en el model
X_test = preprocess_input(X_test)

In [ ]:
preds = base_model.predict(X_test)

In [ ]:
len(preds)

Parece que tenemos 8 predicciones para nuestras imágenes. Pero cada una tiene 1000 potenciales categorías.

In [ ]:
len(preds[0])

Necesitamos recurrir a las categorías con las que fue entrenada la red original para saber qué significan.

In [ ]:
from tensorflow.keras.applications.resnet_v2 import decode_predictions

decodes = decode_predictions(preds, top=3)
decodes

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer que entrenaremos para nuestro ejercicio base de **Perros y Gatos**.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=32

TRAIN_PATH = "data/dogsandcats/train"
filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df.head()

Incluimos el generador para incorporar las variaciones necesarias para que aprenda gatos y perros en cualquier pose, zoom, etc.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(IM_SIZE,IM_SIZE,3),
                    include_top=False, # No añadimos la parte del clasificador
                    weights = 'imagenet'
                )

base_model.summary()

Ahora nos toca añadir el modelo de clasificación que funcionará sobre las características identificadas clave por la red VGG16 (vectores de 512 características).

In [ ]:
from keras import layers
import tensorflow as tf

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.25
x = layers.Dropout(0.25)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)
    
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])
model.summary()

In [ ]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    batch_size = 128,
                    epochs = 20)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(vgghist.history['loss'], color='b', label="Training loss")
ax1.plot(vgghist.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 5, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(vgghist.history['acc'], color='b', label="Training accuracy")
ax2.plot(vgghist.history['val_acc'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 5, 1))

plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

## Hubs de modelos

Basados en este principio han prolifero los sitios donde la gente cuelga sus modelos, siendo uno de los más habituales HuggingFace: https://huggingface.co/models

![](../assets/images/huggingface.png)

Gracias a estos repositorios, podemos acceder a modelos entrenados con grandes volúmenes de datos y posteriormente especializarlos para la tarea que nos convenga a nosotros si esta fuera más específica o diferente de la tarea original. Un poco como el caso de los perros y gatos. Sin duda un ahorro de tiempo y costes relevante.

Podemos ver como ejemplo la red que hemos importado anteriormente, ResNet50, con la información de detalle y ejemplos que muestra Microsoft: https://huggingface.co/microsoft/resnet-50
